# Outlier Rejection

In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt 

def performRejection(ls, lim):
    mean = np.mean(ls)
    stdev = stats.tstd(ls)
    ct = 0
    for i in ls:
        if np.abs(mean - i) > 3*stdev:
            ls.remove(i) 
            ct += 1
    if ct <= lim:
        return ls
    else:
        return performRejection(ls, lim)

In [3]:
def outlierRejection(data, truth, lim):
    data.tolist()
    truth.tolist()
    if len(data) != len(truth):
        print("The dataset and the truth dataset are not the same size! :(")
    else:
        diff = data - truth
        not_outliers = performRejection(diff, lim)
        outlier_rate = ((len(truth) - len(not_outliers))/len(truth))*100
        hist = stats.histogram(not_outliers, bins = 100)
        stdev = stats.tstd(hist[1])

        inds = []
        for i in range(len(not_outliers)):
            inds.append(np.where(diff == not_outliers[i])[0][0])

        kept_y = []
        kept_x = []
        for i in inds:
            kept_y.append(data[i])
            kept_x.append(truth[i])
        np.asarray(kept_y)
        np.asarray(kept_x)

        plt.scatter(truth, data, color = 'cyan', s = len(data)/(10**6))
        plt.scatter(kept_x, kept_y, color = 'blue', s = len(data)/(10**6))
        arr = np.arange(0, len(data))
        plt.plot(arr, arr, color = 'black')

        return outlier_rate, stdev


# Mean vs Mode 

In [ ]:
## in this context, grid is the grid over which we evaluate our pdfs, ens is the statistical ensemble
## ens should be an mxn matrix 
## the length of grid is the number of columns in ens 

import matplotlib.pyplot as plt 
from matplotlib import cm 
from scipy import stats 

def MeanvsMode(grid, ens):
    means = []
    stdevs = []
    for i in range(0, len(ens)): 
        mean = np.sum(grid * ens[i]) / np.sum(ens[i])
        var = np.sum((mean - grid)**2 * ens[i]) / np.sum(ens[i])
        stdev = np.sqrt(var)
        means.append(mean)
        stdevs.append(stdev)
    np.asarray(means) 
    np.asarray(stdevs)

    modes = []
    for i in range(0, len(ens)):
        mode = np.max(ens[i])
        modes.append(mode)
    np.asarray(modes)

    arr = np.arange(0, np.max(np.max(means), np.max(modes)), 0.1)
    
    dists = []
    for i in range(len(means)):
        dist = np.abs(means[i] - modes[i]) / np.sqrt(2)
        dists.append[dist]
    outlier_ct = 0
    for i in dists:
        if i >= 5*stats.tstd(dists):
            outlier_ct += 1
    outlier_rate = (len(modes) - outlier_ct) / len(modes)

    plt.scatter(modes, means, s=1, c = stdevs, cmap = cm.plasma)
    plt.plot(arr, arr, color = "skyblue")
    plt.xlabel('Mode')
    plt.ylabel('Mean')
    plt.colorbar(label = 'Standard Deviation')
    plt.title("Gaussianity Plot", fontsize = 'large', fontweight = 'bold')

    return outlier_rate

# PIT

In [ ]:
import qp

## prob wont work 
def PIT1(data, truth):
    from qp.metrics.pit import PIT
    pitobj = PIT(data(), truth)
    metamets = pitobj.calculate_pit_meta_metrics()
    norm_pit_vals = np.array(pitobj.pit_samps) / len(data)
    pit_out_rate = metamets['outlier_rate']
    print(f"PIT outlier rate of this sample: {pit_out_rate:.6f}") 
    pit_out_rate = pitobj.evaluate_PIT_outlier_rate()
    print(f"PIT outlier rate of this sample: {pit_out_rate:.6f}") 
    plt.hist(norm_pit_vals, bins = 100)

    quant_ens = pitobj.pit
    obj = quant_ens.objdata()
    meta = quant_ens.metadata()
    locs = obj['locs']
    quants = meta['quants']
    arr = np.arange(0, 1, 0.1)
    plt.plot(locs[0], quants[0], color = 'orange')
    plt.plot(arr, arr, color = 'black')

In [ ]:
def PIT2(datafile, truth):
    from qp.metrics.pit import PIT
    test_pit = PIT(datafile(), truth)
    eval_pit = test_pit._pit_samps
    np.save('pit_output.npy', eval_pit)
    eval_pit = np.load('pit_output.npy')

    PIThist = plt.hist(eval_pit, bins = 100, density = True)
    outlier_rate = (PIThist[0][0] + PIThist[0][-1]) / np.sum(PIThist[0]) 

    from scipy.stats import uniform 
    plt.semilogy()
    uniform_dist = plt.plot(PIThist[1], np.ones(len(PIThist[0])+1), label = "catastrophic outlier rate:" + str(outlier_rate), color = 'white')
    plt.plot(PIThist[1], uniform().pdf(PIThist[1]), color = 'orange')
    plt.title("Histogram of PIT", fontweight = 'bold')
    plt.xlabel('$\int_{-\infty}^{z_{true}} \! \hat{p}(z|photometry) \, dz $', fontsize = 'large')
    plt.ylabel('Bin Count', fontsize = 'large')
    plt.legend(handles = uniform_dist)

    return outlier_rate